In [2]:
import arcpy
from dataclasses import dataclass
from archaic import FeatureClass

arcpy.env.workspace = ".data/canada.geodatabase"  # type: ignore


@dataclass(frozen=True)
class Mine:
    oid: int
    name_e: str
    type_e: str
    shape: arcpy.Multipoint


feature_class = FeatureClass[Mine]("main.mines_pt")

for mine in feature_class.read([1, 2, 7]):
    print(mine)

feature_class.info.__dict__

Mine(oid=1, name_e='Victor', type_e='Diamond', shape=<PointGeometry object at 0x186a0c1afa0[0x186a0d22990]>)
Mine(oid=2, name_e='Musselwhite', type_e='Gold', shape=<PointGeometry object at 0x186a0d22820[0x186a0be1900]>)
Mine(oid=7, name_e='Bucko Lake', type_e='Nickel', shape=<PointGeometry object at 0x186a0c1afa0[0x186a0d22990]>)


{'model': __main__.Mine,
 'catalog_path': '.data/canada.geodatabase\\main.mines_pt',
 'oid_field_name': 'OBJECTID',
 'oid_property_name': 'oid',
 'has_parameterless_constructor': False,
 'properties': {'oid': 'OID@',
  'name_e': 'NAME_E',
  'type_e': 'TYPE_E',
  'shape': 'SHAPE@'},
 'edit_properties': {'oid': 'OID@',
  'name_e': 'NAME_E',
  'type_e': 'TYPE_E',
  'shape': 'SHAPE@'},
 '__orig_class__': archaic.info.Info[~T]}